In [2]:
#r "nuget: Shioaji, 1.1.8"

Installed Packages Shioaji, 1.1.8

In [3]:
using System.IO;
using Sinopac.Shioaji;
using System.Text.Json;
Shioaji _api = new Shioaji();

string jsonString = File.ReadAllText(@"D:\DotnetReactShioaji\DotnetReactShioaji\Sinopac.json");
JsonElement root = JsonDocument.Parse(jsonString).RootElement;
_api.Login(root.GetProperty("API_Key").GetString(), root.GetProperty("Secret_Key").GetString());

// var res = _api.Snapshots(new List<IContract>() {_api.Contracts.Futures["TXF"]["TXFR1"]}) //若誤用[0]就導它無法在array裡搜，硬用linq就報錯

Connecting as shioaji@sinopac on 210.59.255.161:80...


In [4]:
public dynamic anonListProfitLossSummary(string bgn, string end)
{
    var src = _api.ListProfitLossSummary(bgn, end, _api.FutureAccount).profitloss_summary[0];
    var productAnonymous = new
    {
        code = (src.code.Substring(3, 5)) + (src.code[8] < 'L' ? "Call" : "Put"),
        direction = src.direction,
        entry_price = src.entry_price,
        cover_price = src.cover_price,
        netPnl = src.pnl - src.fee - src.tax
    };
    return productAnonymous;
}

List<object> _temp = new List<object>();
_temp.Add(anonListProfitLossSummary("2024-01-10", "2024-01-10"));
_temp.Add(anonListProfitLossSummary("2024-01-18", "2024-01-18"));
// _temp.Add(anonListProfitLossSummary("2024-01-31", "2024-01-31"));
// _temp.Where(x => (Decimal)x.GetType().GetProperty("entry_price").GetValue(x) >= (Decimal)49.0)

_temp//.Max(x => (Decimal)x.GetType().GetProperty("entry_price").GetValue(x))

index value 0 { code = 17550Put, direction = Buy, entry_price = 49.0, cover_price = 17481.0, netPnl = 951.0 } code 17550Put direction Buy entry_price 49.0 cover_price 17481.0 netPnl 951.0 1 { code = 17250Call, direction = Buy, entry_price = 99.0, cover_price = 125.0, netPnl = 1259.0 } code 17250Call direction Buy entry_price 99.0 cover_price 125.0 netPnl 1259.0

In [42]:
List<Sinopac.Shioaji.FuturePosition> _src = _api.ListPositions(_api.FutureAccount);  //允許多筆故idx 0會報錯
// _src.Where(x => x.code[8]<'L').Select(x=>x.code)
IEnumerable data;
if(_src.Any(x => x.code[8] > 'L'))
{
    data = _src.Select(x=>x.code);
}
else
{
    Console.WriteLine("待建BCSC的部位"); 
}
data
// var direction = _src.Where(x => x.code.Contains("TX5")).Select(x => x.direction).FirstOrDefault();
// var entry_price = _src.Where(x => x.code.Contains("TX5")).Select(x => x.price).FirstOrDefault();
// var last_price = _src.Where(x => x.code.Contains("TX5")).Select(x => x.last_price).FirstOrDefault();
// var Pnl = _src.Where(x => x.code.Contains("TX5")).Select(x => x.pnl).FirstOrDefault();
// Pnl

[ TX117900N4, TX118000N4 ]

In [63]:
_api.ListPositions(_api.FutureAccount)

index value 0 {\nid=0, code=TX117900N4, direction=Sell, quantity=1, price=129.0, last_price=107.0, pnl=1100.0\n} id 0 code TX117900N4 direction Sell quantity 1 price 129.0 last_price 107.0 pnl 1100.0 1 {\nid=1, code=TX118000N4, direction=Buy, quantity=1, price=176.0, last_price=149.0, pnl=-1350.0\n} id 1 code TX118000N4 direction Buy quantity 1 price 176.0 last_price 149.0 pnl -1350.0

In [71]:
List<Sinopac.Shioaji.FuturePosition> _src = _api.ListPositions(_api.FutureAccount);
if(_src.Any(x => x.code.Contains("MXF")))
{
    //準備監控 移停、出場或停損
}
else
{
    if (_src.Any(x => x.code[8] < 'L'))
    {
        var data1 =_src.Select(x=> new {x.price, x.last_price, x.pnl});
        Console.WriteLine(string.Join(", ", data1));
    }
    else if(_src.Any(x => x.code[8] > 'L'))
    {
        var data1 =_src.Select(x=> new {x.price, x.last_price, x.pnl});
        Console.WriteLine(string.Join(", ", data1));
    }
    else
    {
        Console.WriteLine("空手沒部位"); 
    }
}

{ price = 129.0, last_price = 106.0, pnl = 1150.0 }, { price = 176.0, last_price = 147.0, pnl = -1450.0 }


In [ ]:
using System.Threading;

public void MxfMock(double argEntryPrice, double argStp)
{
    List<double> _temp = new List<double>();
    while (true)
    {
        double spotClose = (double)_api.Snapshots(new List<IContract>() { _api.Contracts.Futures["TXF"]["TXFR1"] })[0].close;
        _temp.Add(spotClose);
        _temp.Reverse();
        string result = string.Join(", ", _temp.Take(10));
        Console.WriteLine($"現在{spotClose}點 \t 過去 10 ticks {result}\n高點{_temp.Max()} since start");
        if (spotClose >= argEntryPrice + argStp)
        {
            argEntryPrice += 2;
            Console.WriteLine($"已拉開故到{argEntryPrice}才會被掃出場");
            while (true)
            {
                spotClose = (double)_api.Snapshots(new List<IContract>() { _api.Contracts.Futures["TXF"]["TXFR1"] })[0].close;
                _temp.Add(spotClose);
                if (spotClose <= (_temp.Max() - argStp))
                {
                    string desc = spotClose >= argEntryPrice ? "停利" : "保本";
                    Console.WriteLine($"★{desc}出在{spotClose}。");
                    break;
                }
                else if (spotClose <= argEntryPrice)
                {
                    Console.WriteLine($"★保本出在{spotClose}。");
                    break;
                }
                else
                {
                    Console.WriteLine(spotClose);
                    Thread.Sleep(5_000);
                }
            }
            break;
        }
        else if (spotClose <= argEntryPrice - argStp)
        {
            Console.WriteLine($"★停損，出在{spotClose}。");
            break;
        }
        else Thread.Sleep(5_000);
    }
    Console.WriteLine("★★小台積期：" + _api.Snapshots(new List<IContract>() { _api.Contracts.Futures["QFF"]["QFFR1"] })[0].close);
}


DateTime DTN = DateTime.Now;
bool preMktTime = DTN < DTN.Date + new TimeSpan(5, 00, 0);
bool onMktTime = (DTN > DTN.Date + new TimeSpan(8, 45, 0) && DTN < DTN.Date + new TimeSpan(13, 45, 0));
DateTime end = preMktTime ? DTN.Date + new TimeSpan(5, 00, 0) : (onMktTime ? DTN.Date + new TimeSpan(13, 45, 0) : DateTime.Now.Date + new TimeSpan(29, 0, 0));
while (DateTime.Now < end )
{
    // 1.有部位就依成本價
    // List<Sinopac.Shioaji.FuturePosition> _src = _api.ListPositions(_api.FutureAccount);
    // var Pnl = _src.Where(x => x.code.Contains("MXF")).Select(x => x.pnl).FirstOrDefault();
    // var entry_price = _src.Where(x => x.code.Contains("MXF")).Select(x => x.price).FirstOrDefault();
    

    // if(Pnl >= entry_price)
    // {
    //     MxfMock( rtPositions.First().price, decimal 10.0)
    // }
    if(false)
    {
        //
    }


    // 2.無部位就主觀抓底點
    else
    {
        MxfMock(17975.0, 10.0);
        break;
    }    
}

In [34]:
_api.Contracts.Futures["QFF"]["QFFR1"]

security_type,FUT
code,QFFR1
category,QFF
symbol,QFFR1
exchange,TAIFEX
limit_up,691
limit_down,567
reference,629
update_date,2024/02/01
name,小型台積電期貨近月
target_code,QFFB4


In [ ]:
// var _contract = _api.Contracts.Futures["MXF"]["MXF202402"];    
// var _futOptOrder = new FutOptOrder()
// {
//     action = "Buy",
//     price = 18000,
//     quantity = 1, price_type = "LMT", order_type = "ROD"//, octype = OCType.DayTrade
// };
// var _trade = _api.PlaceOrder(_contract, _futOptOrder);
// _trade.GetType()
foreach(var i in _trade.status)
{
    Console.WriteLine(i);
}

In [ ]:
// _trade.order
foreach(var i in _trade.order)
{
    Console.WriteLine(i[0].value);
}